In [ ]:
# for feature extraction of test images

In [ ]:
import tensorflow as tf
import numpy as np
import json
from tqdm import tqdm
import os, sys, shutil
import h5py
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
main_dir = # PATH TO THE SHARED FOLDER

In [ ]:
images_dir = os.path.join(main_dir, 'images_test/test_images')
%cd {images_dir}

In [ ]:
# if taking too long, wait until it timeouts and try again until it's faster. Proceed after that
Image.open(os.path.join(images_dir, '0.jpg')).resize((224,224))

In [ ]:
# see how many images by using this code. Everything loaded when len(images) becomes 35094.
images = []
with os.scandir(images_dir) as it:
  for entry in tqdm(it):
    if not entry.name.startswith('.') and entry.is_file():
      images.append(entry)

In [ ]:
test_dest_paths = np.loadtxt(os.path.join(main_dir, 'test_dest_paths.txt'), dtype=str)
test_image_paths = np.loadtxt(os.path.join(main_dir, 'test_image_paths.txt'), dtype=str)

In [ ]:
resnet50 = tf.keras.applications.resnet50.ResNet50(weights='imagenet')
feature_extr = tf.keras.Model(resnet50.input,resnet50.layers[-2].output)

In [ ]:
# start with these, if runtime disconnects or you stop the cell, say cur_img = the value you see in progress bar
n_ims = len(test_image_paths)
cur_img = 0
rest = n_ims - cur_img

In [ ]:
# make sure GPU is on 
with tf.device('/device:GPU:0'):
  for img, dest in tqdm(zip(test_image_paths[cur_img:], test_dest_paths[cur_img:]), total=rest): 
    im = image.load_img(img, target_size=(224, 224))
    x = image.img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    feature = feature_extr.predict(x)
    feature = np.reshape(feature, feature.shape[1])
    np.savetxt(dest, feature, fmt='%s')